In [ ]:
from pyspark.sql.functions import col, udf, lit

In [ ]:
#conexao
blob_account_name = "laketeste"
blob_container_name = f"bronze2"
blob_relative_path = f"/"
blob_sas_token = r"rvjwdhckOb3bwcsN7vqxeXjrXkclvImADgD8l0H20h1RVmU2iCWZKhI3QDPrEkTyuUdvFn2N27dlxRM0dRInOg=="

In [ ]:
#conexao com o blob
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)
print('Remote blob path: ' + wasbs_path)

In [ ]:

# Lendo todos arquivos na pasta e guardando em uma lista
folder = dbutils.fs.ls(wasbs_path)
dataframes = []
for file in folder:
  csv_file = spark.read.option("header", "true").option("mode", "PERMISSIVE").csv(file[0],  sep=';')
  dataframes.append(csv_file)
  display(csv_file)

In [ ]:
#Pegando o nome dos arquivos e guardando em uma lista
list_nomes = []
for i in range(len(folder)):
  csv = folder[i][1]
  test = csv.replace(".csv", "")
  final = test.replace("-", "")
  list_nomes.append(final)
  print(final)

In [ ]:
#info do banco de dados

jdbcHostname = "testeserv.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "databaseteste"
username =  "user"
password = "password"

In [ ]:
#conexão com o banco de dados

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
print(jdbcUrl)
connectionProperties = {
  "user" : username,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
} 


In [ ]:
#Loop para escrever os dataframes no SQL Database
for i in range(len(folder)):
  myfinaldf = DataFrameWriter(dataframes[i])
  myfinaldf.jdbc(url=jdbcUrl, table= list_nomes[i], mode ="overwrite", properties = connectionProperties)
  print(dataframes[i])